In [ ]:
import os
import glob
import sys
import numpy as np
import datetime as dt
from multiprocessing import Pool
from tqdm.notebook import tqdm
import gc

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
force = False

lag_dir = 'NERSC_arctic25km_sea_ice_age_v2p1'
dst_datetimes = [dt.datetime(2023, 1, 1) + dt.timedelta(i) for i in range(0, 943, 1)]

dst_dir = f'{lag_dir}/age'
print(len(dst_datetimes), dst_datetimes[0], dst_datetimes[-1])

In [ ]:
def compute_age(dst_datetime):
    dst_date = dst_datetime.strftime('%Y%m%d')
    dst_date_dir = f'{dst_dir}/{dst_datetime.strftime("%Y")}'
    os.makedirs(dst_date_dir, exist_ok=True)
    dst_npz_file = f'{dst_date_dir}/age_{dst_date}.npz'

    if os.path.exists(dst_npz_file):
        return

    dst_sic_file = glob.glob(f'{lag_dir}/sic/*/sic_{dst_date}.npz')[0]
    try:
        c = np.load(dst_sic_file)['c']
    except:
        print(f'BAD SIC file {dst_sic_file}')
        raise

    dst_mesh_file = glob.glob(f'{lag_dir}/mesh/*/mesh_{dst_date}.npz')[0]
    try:
        x = np.load(dst_mesh_file)['x']
        y = np.load(dst_mesh_file)['y']
        t = np.load(dst_mesh_file)['t']
    except:
        print(f'BAD MESH file {dst_mesh_file}')
        raise

    src_sic_files = sorted(glob.glob(f'{lag_dir}/sic/*/sic_*_{dst_date}.npz'))[-6:]
    # for osisaf/topaz/nanuk intercomparison only:
    #src_sic_files = [f for f in src_sic_files if int(os.path.basename(f).split('_')[1][:4]) >= 2005]

    src_sic = []
    for src_sic_file in src_sic_files:
        try:
            src_c = np.load(src_sic_file)['c']
        except:
            print(f'BAD SIC file {src_sic_file}')
            raise
        else:
            src_sic.append(src_c)

    fractions = np.diff(np.array([np.zeros_like(c)] + src_sic + [c]), axis=0)
    nfracs = fractions.shape[0]

    if dt.datetime(4,dst_datetime.month,dst_datetime.day) >= dt.datetime(4,9,15):
        dst_age_offset = dst_datetime - dt.datetime(dst_datetime.year, 9, 15)
    else:
        dst_age_offset = dst_datetime - dt.datetime(dst_datetime.year-1, 9, 15)
    dst_age_offset = dst_age_offset.days / 365.

    years = np.arange(nfracs, 0, -1)
    years = years - 1 + dst_age_offset
    age = np.sum(fractions * years[None].T, axis=0)/100.
    np.savez(dst_npz_file,
             a=age.astype(np.float32),
             c=c.astype(np.float32),
             x=x.astype(np.float32),
             y=y.astype(np.float32),
             t=t.astype(np.float32),
             f=fractions.astype(np.float32))

for dst_datetime in tqdm(dst_datetimes):
    compute_age(dst_datetime)


In [ ]:
age_files = sorted(glob.glob(f'{dst_dir}/202*/age_????????.npz'))[::5]
age_indices = list(range(len(age_files)))
frame_files = sorted(glob.glob(f'{dst_dir}/frame_????.png'), reverse=True)
frame_indices = [int(os.path.basename(f).split('.')[0].split('_')[1]) for f in frame_files]
_ = [age_files.pop(i) for i in frame_indices]
_ = [age_indices.pop(i) for i in frame_indices]
print(len(age_files), len(age_indices), age_files[0], age_files[-1])

In [ ]:
cmap = 'jet'
vmin = 0
vmax = 3

def plot_age(i_age_file):
    i, age_file = i_age_file
    dst_frame_name = f'{dst_dir}/frames/frame_{i:04}.png'
    if os.path.exists(dst_frame_name):
        return
    try:
        d = dict(np.load(age_file))
    except:
        print(f'ERROR reading {age_file}')
        return

    fig = plt.figure(figsize=(10,10))
    try:
        plt.tripcolor(d['x'], d['y'], d['a'], triangles=d['t'], cmap=cmap, vmin=vmin, vmax=vmax)
    except:
        print(f'ERROR in {i_age_file}')
    else:
        plt.colorbar(shrink=0.5)
        date_str = os.path.basename(age_file).split('_')[1].split('.')[0]
        text = f'#{i:04} {date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}'
        plt.text(-2000, 2200, text, fontsize=16)
        plt.axis('off')
        plt.tight_layout()
    plt.savefig(dst_frame_name, bbox_inches='tight', pad_inches=0, facecolor='white')
    plt.close('all')
    plt.close()
    gc.collect()


for i_age_file in tqdm(zip(age_indices, age_files), total=len(age_files)):
    plot_age(i_age_file)


In [ ]:
!ffmpeg -y -r 10 -f image2 -i ./NERSC_arctic25km_sea_ice_age_v2p1/age/frames/frame_%04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -vcodec libx264 -crf 1 -pix_fmt yuv420p ./NERSC_arctic25km_sea_ice_age_v2p1/age/frames/age_2020.mp4